In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import squarify
import plotly.express as px



# **CATEGORIES:**


In [504]:
categories = pd.read_csv('categories_clean.csv')

In [505]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category_id      176 non-null    int64 
 1   category_name    176 non-null    object
 2   parent_category  176 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.3+ KB


LIMPIEZA DE LAS COLUMNAS Y DE DATOS

In [506]:
categories.drop(columns=['Unnamed: 0','created_at'], errors = 'ignore', inplace = True)

#Reduzco a un int32 category_id, ya que la cantidad de valores que puedo abarcar con esos 32bits es suficiente para categorizar productos.
categories['category_id'] = categories['category_id'].astype(np.int32)

# recorro cada dato de ambas col y las paso a string para manejarlas mas facil y acomodar los datos.
for col in ['parent_category', 'category_name']:
  categories[col] = categories[col].astype('string').str.strip().str.title().replace(['nan','undefined','','NaN','Undefined'],pd.NA).fillna('Unknown')

# hago una mascara para solo quedarme los datos que no tengan ambas col como unknown.
mask = (categories['parent_category'] == 'Unknown') & (categories['category_name'] == 'Unknown')
categories = categories[~mask].copy()

categories["category_name"] = categories['category_name'].astype('category')
categories['parent_category'] = categories['parent_category'].astype('category')

#categories_in_order = categories.set_index(["parent_category","category_name"]).sort_index(ascending=True)


In [507]:
categories.loc[categories["category_id"] == 9]


,category_id,category_name,parent_category
8,9,Children'S Clothing,Clothing


# **CUSTOMERS**

In [508]:
customers = pd.read_csv('customers_clean.csv')

In [509]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   customer_id        500000 non-null  int64 
 1   email              500000 non-null  object
 2   first_name         500000 non-null  object
 3   last_name          500000 non-null  object
 4   date_of_birth      454914 non-null  object
 5   gender             500000 non-null  object
 6   country            500000 non-null  object
 7   city               500000 non-null  object
 8   postal_code        500000 non-null  object
 9   address            500000 non-null  object
 10  registration_date  455155 non-null  object
 11  last_login         454954 non-null  object
 12  is_active          500000 non-null  bool  
 13  customer_segment   500000 non-null  object
 14  marketing_consent  500000 non-null  bool  
dtypes: bool(2), int64(1), object(12)
memory usage: 50.5+ MB


In [510]:
customers.drop(columns=['Unnamed: 0','phone'], errors = 'ignore', inplace = True)

#limpio columna gender
customers['gender'] = customers['gender'].astype('string').str.strip().str.upper().fillna('OTHER').replace('UNDEFINED','OTHER').astype('category')

# generos = customers.groupby('gender').agg(cantidad=('gender','count'))

#limpio y arreglo country
customers['country'] = customers['country'].astype('string').str.strip().str.title().fillna('Undefined').astype('category')

#limpio y arreglo city
customers['city'] = customers['city'].astype('string').str.strip().str.title().fillna('-').replace('Undefined','-')


#arreglo CP
customers['postal_code'] = customers['postal_code'].astype('string').str.strip().fillna('Undefined').replace('Undefined','-') #arreglar cp

#arreglo addres
customers['address'] = customers['address'].astype('string').str.strip().fillna('Undefined').replace('Undefined','-')

#arreglo email
customers['email'] = customers['email'].astype('string').str.strip().str.lower().fillna('Undefined').replace('Undefined','-')

#arreglo nombre
customers['first_name'] = customers['first_name'].astype('string').str.strip().str.title().fillna('Undefined').replace('Undefined','-')

#arreglo apellido
customers['last_name'] = customers['last_name'].astype('string').str.strip().str.title().fillna('Undefined').replace('Undefined','-')

#arreglo fecha de nacimiento
customers['date_of_birth'] = customers['date_of_birth'].astype('string').str.strip().replace('', pd.NA)      # si queda vacío, lo marca como NA
customers['date_of_birth'] = pd.to_datetime(customers['date_of_birth'], errors='coerce')

#arreglo customer_Segment
customers['customer_segment'] = customers['customer_segment'].astype('string').str.strip().str.title().fillna('Undefined').replace('Undefined','Others').astype('category')

#arreglo registration date y last login con formato datetime
customers['registration_date'] = customers['registration_date'].astype('string').str.strip().replace('', pd.NA)
customers['registration_date'] = pd.to_datetime(customers['registration_date'],errors='coerce')

customers['last_login'] = customers['last_login'].astype('string').str.strip().replace('', pd.NA)
customers['last_login'] = pd.to_datetime(customers['last_login'],errors='coerce')


In [511]:
customers

,customer_id,email,first_name,last_name,date_of_birth,gender,country,city,postal_code,address,registration_date,last_login,is_active,customer_segment,marketing_consent
0,1,uthomas@example.net,Kayla,Smith,1948-05-20,F,Brazil,South Michelle,70351.0,-,2022-09-22 09:40:47.913063,2025-01-07 12:09:24.323425,True,Regular,True
1,2,caldwellandrew@example.com,Carolyn,Gray,1983-03-19,F,France,Lake Shaneville,65372.0,"247 Allison Overpass Suite 960 North Dillon, A...",2022-11-14 23:20:08.324356,2024-08-25 17:36:03.417619,True,Budget,False
2,3,jon32@example.org,Deborah,Martin,1940-05-23,M,Usa,Lisaborough,32855.0,"61358 Mills Spur Lake Tiffany, MS 52073",2022-10-14 15:49:17.728606,2023-04-10 06:15:20.412198,True,Budget,True
3,4,ksmith@example.org,Rachel,Ellison,1969-12-21,M,Australia,South Michaelborough,11530.0,undefined,2024-03-16 09:18:53.251566,2025-03-20 12:06:38.326651,True,Others,False
4,5,andrewsjessica@example.net,Lisa,Phelps,1940-05-18,M,Japan,Herrerabury,61228.0,"6184 King Trail Lake Trevor, IA 55611",2023-07-21 10:07:22.968497,2025-04-19 11:40:01.441135,True,Regular,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499996,adampeterson@example.com,Kara,Smith,1942-07-29,OTHER,Mexico,North Donald,83228.0,-,2022-12-08 21:37:25.538388,NaT,True,Premium,False
499996,499997,walshdevon@example.org,Christopher,Peterson,1999-12-21,F,Australia,-,58730.0,USNV Sherman FPO AA 42084,2023-06-28 04:04:19.964403,NaT,True,Regular,True
499997,499998,freemanlisa@example.com,Joshua,Macdonald,1985-06-16,OTHER,Uk,West Joshuaborough,2742.0,"96708 Fowler Island Kristinton, OR 21429",2024-04-13 12:31:54.977860,2024-10-16 17:13:41.627636,False,Others,False
499998,499999,preese@example.com,Jonathan,Brown,2006-07-21,M,Undefined,Robinsonville,-,-,2022-08-26 11:04:41.091918,2025-03-08 17:06:18.286590,False,Regular,True


In [512]:
customers['country'].unique()

['Brazil', 'France', 'Usa', 'Australia', 'Japan', ..., 'India', 'Mexico', 'Undefined', 'Canada', 'Uk']
Length: 11
Categories (11, string): [Australia, Brazil, Canada, France, ..., Mexico, Uk, Undefined, Usa]

# **INVENTORY LOGS**

In [513]:
inventory_logs = pd.read_csv('inventory_logs_clean.csv')

In [514]:
inventory_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       400000 non-null  int64 
 1   log_id           400000 non-null  int64 
 2   product_id       400000 non-null  int64 
 3   movement_type    400000 non-null  object
 4   quantity_change  400000 non-null  int64 
 5   reason           400000 non-null  object
 6   timestamp        363851 non-null  object
 7   reference_id     400000 non-null  int64 
 8   notes            400000 non-null  object
dtypes: int64(5), object(4)
memory usage: 27.5+ MB


In [515]:
inventory_logs['log_id'] = inventory_logs['log_id'].astype(np.int32)
inventory_logs['product_id'] = inventory_logs['product_id'].astype(np.int32)
inventory_logs['reference_id'] = inventory_logs['reference_id'].astype(np.int32)


In [516]:
inventory_logs['movement_type'] = inventory_logs['movement_type'].astype('string').str.strip().str.title().fillna('Undefined').astype('category')

inventory_logs['reason'] = inventory_logs['reason'].astype('string').str.strip().str.title().fillna('Other').replace('Undefined','Other').astype('category')

inventory_logs['timestamp'] = inventory_logs['timestamp'].astype('string').str.strip().replace('', pd.NA)
inventory_logs['timestamp'] = pd.to_datetime(inventory_logs['timestamp'],errors='coerce')

inventory_logs['notes'] = inventory_logs['notes'].astype('string').str.strip().fillna('-')

#aca modifico los Nan ya que en mi contexto no causaron ninguna modificacion en el inventario
inventory_logs['quantity_change'] = inventory_logs['quantity_change'].fillna(0).astype(np.int32)


In [517]:
inventory_logs

,Unnamed: 0,log_id,product_id,movement_type,quantity_change,reason,timestamp,reference_id,notes
0,0,1,996381,In,228,Adjustment,2023-08-21 04:33:50.446753,57097,-
1,1,2,962832,Out,0,Return,2024-09-25 06:15:03.583328,37909,-
2,2,3,995198,Out,0,Sale,2024-05-06 01:11:03.276581,18823,-
3,3,4,997550,In,260,Adjustment,2024-09-03 20:47:37.637720,80650,Director own ball land themselves key.
4,4,5,978028,In,263,Return,2024-10-19 08:55:45.508647,90031,-
...,...,...,...,...,...,...,...,...,...
399995,399995,399996,919038,In,0,Transfer,2025-02-11 18:36:04.636086,72666,-
399996,399996,399997,963725,Adjustment,0,Purchase,2024-01-24 08:17:19.002028,57349,-
399997,399997,399998,958015,Adjustment,-60,Theft,2024-05-27 12:12:05.392410,15080,Make pass including do unit standard stand water.
399998,399998,399999,937235,Out,0,Return,2025-05-15 14:57:14.808283,46924,-


# **ORDER_ITEMS**

In [518]:
order_items = pd.read_csv('order_items_clean.csv')

In [519]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285243 entries, 0 to 285242
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   order_item_id    285243 non-null  int64  
 1   order_id         285243 non-null  int64  
 2   product_id       285243 non-null  int64  
 3   quantity         285243 non-null  int64  
 4   unit_price       242457 non-null  float64
 5   line_total       242457 non-null  float64
 6   discount_amount  285243 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 15.2 MB


In [520]:
order_items.drop(columns=['Unnamed: 0'], errors = 'ignore', inplace = True)

order_items['order_item_id'] = order_items['order_item_id'].astype(np.int32)
order_items['order_id'] = order_items['order_id'].astype(np.int32)
order_items['product_id'] = order_items['product_id'].astype(np.int32)
order_items['quantity'] = order_items['quantity'].fillna(0).astype(np.int32)
order_items['discount_amount'] = order_items['discount_amount'].fillna(0)



In [521]:
order_items

,order_item_id,order_id,product_id,quantity,unit_price,line_total,discount_amount
0,1,19900399,940377,6,569.28,3415.68,0.00
1,2,19900399,935931,2,1090.20,NaN,0.00
2,3,19900399,905111,7,NaN,1176.98,154.52
3,4,19900399,920065,3,464.45,1393.35,0.00
4,5,19900400,927289,8,283.12,NaN,0.00
...,...,...,...,...,...,...,...
285238,285239,19999999,912297,1,88.40,88.40,0.00
285239,285240,19999999,945504,10,1435.85,14358.50,0.00
285240,285241,20000000,972378,8,172.43,1379.44,158.71
285241,285242,20000000,933150,1,NaN,811.86,0.00


# **ORDERS**





In [4]:
# orders = pd.read_csv('orders.csv')
orders = pd.read_csv('orders_clean.csv')

/tmp/ipython-input-1722490415.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('orders_clean.csv')


In [ ]:
orders.info()

In [ ]:
orders

In [ ]:
#orders.drop(columns=['Unnamed: 0'], errors = 'ignore', inplace = True)

orders['order_id'] = orders['order_id'].astype(np.int32)
orders['customer_id'] = orders['customer_id'].astype(np.int32)
orders['order_date'] = orders['order_date'].astype('string').str.strip().replace('', pd.NA)
orders['order_date']  = pd.to_datetime(orders['created_at'],errors='coerce')
orders['status'] = orders['status'].astype('string').str.strip().str.title().fillna('Undefined').astype('category')
orders['payment_method'] = orders['payment_method'].astype('string').str.strip().str.title().fillna('Other').replace('Undefined','Other').astype('category')

orders['shipping_address'] = orders['shipping_address'].astype('string').str.strip().fillna('-')
orders['billing_address'] = orders['billing_address'].astype('string').str.strip().fillna('-')

In [ ]:

orders['currency'] = orders['currency'].astype('string').str.strip().str.upper().replace('UNDEFINED',pd.NA).astype('category')

orders['created_at'] = orders['created_at'].astype('string').str.strip().replace('', pd.NA)
orders['updated_at'] = orders['updated_at'].astype('string').str.strip().replace('', pd.NA)      # si queda vacío, lo marca como NA

orders['created_at'] = pd.to_datetime(orders['created_at'],errors='coerce')
orders['updated_at'] = pd.to_datetime(orders['updated_at'],errors='coerce')

In [ ]:
orders['discount_amount'] = orders['discount_amount'].fillna(0)
orders['tax_amount'] = orders['tax_amount'].fillna(0)
orders['shipping_cost'] = orders['shipping_cost'].fillna(0)



In [ ]:
orders

# **PRODUCTS**

In [ ]:
products = pd.read_csv('products_clean.csv')

In [ ]:
products.info()

In [ ]:
products.drop(columns=['Unnamed: 0'], errors = 'ignore', inplace = True)

products['product_id'] = products['product_id'].astype(np.int32)

products = products.dropna(subset=["category_id"])
products["category_id"] = products["category_id"].astype("int32")

products['stock_quantity'] = products['stock_quantity'].fillna(0).astype(np.int32)

products['brand'] = products['brand'].astype('string').str.strip().str.title().fillna('Undefined').astype('category')
products['weight_kg'] = products['weight_kg'].astype(np.float32)
products['dimensions'] = products['dimensions'].astype('string').str.strip().fillna('-')

products['product_name'] = products['product_name'].astype('string').str.strip().str.title()

products['description'] = products['description'].astype('string').str.strip().fillna('-')

products['created_at'] = products['created_at'].astype('string').str.strip().replace('', pd.NA)
products['created_at'] = pd.to_datetime(products['created_at'],errors='coerce')

In [ ]:
products.brand.unique()

In [ ]:
products

# **REVIEWS**

In [ ]:
reviews = pd.read_csv('reviews_clean.csv')

In [ ]:
reviews.info()

In [ ]:
reviews.drop(columns=['Unnamed: 0'], errors = 'ignore', inplace = True)

reviews['customer_id'] = reviews['customer_id'].astype(np.int32)
reviews['product_id'] = reviews['product_id'].astype(np.int32)

reviews['rating'] = reviews['rating'].astype('Int8')

reviews['title'] = reviews['title'].astype('string').str.strip().fillna('-')
reviews['comment'] = reviews['comment'].astype('string').str.strip().fillna('-')
reviews['helpful_votes'] = reviews['helpful_votes'].fillna(0).astype('Int32')


reviews['created_at'] = reviews['created_at'].astype('string').str.strip().replace('', pd.NA)
reviews['created_at'] = pd.to_datetime(reviews['created_at'], errors='coerce')


In [ ]:
reviews

In [ ]:
# categories.to_csv('categories_clean.csv',index=False)
# customers.to_csv('customers_clean.csv',index=False)
# inventory_logs.to_csv('inventory_logs_clean.csv',index=False)
# order_items.to_csv('order_items_clean.csv',index=False)
# products.to_csv('products_clean.csv',index=False)
# reviews.to_csv('reviews_clean.csv',index=False)
# orders.to_csv('orders_clean.csv', index=False)

# **ENUNCIADO PANDAS**
1) Cuál es el estado que más descuentos tiene en total? y en promedio? Supongan que de una direccion del estilo: 3123 Alan Extension Port Andrea, MA 26926, “MA” es el estado.

2) ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado ‘Refunded’? ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

3) Para cada tipo de pago y segmento de cliente, devolver la suma y el promedio expresado como porcentaje, de clientes activos y de consentimiento de marketing. Se valora que el output de la consulta tenga nombres claros y en español.

4) Para los productos que contienen en su descripción la palabra “stuff” (sin importar mayúsculas o minúsculas), calcular el peso total de su inventario agrupado por marca, mostrar sólo la marca y el peso total de las 5 más pesadas.

# ***1-Cuál es el estado que más descuentos tiene en total? y en promedio? Supongan que de una direccion del estilo: 3123 Alan Extension Port Andrea, MA 26926, “MA” es el estado.

***

In [ ]:
#1
discounts_by_state = orders.copy()


In [ ]:
discounts_by_state

In [ ]:
discounts_by_state['state'] = orders['billing_address'].str.extract(r',\s*([A-Z]{2})\s*\d{5}')


In [ ]:
discounts_by_state[discounts_by_state.state == 'FM']

In [ ]:
discounts_by_state.drop(columns=['order_id','customer_id','order_date',	'status','zip_code',	'payment_method','shipping_address','billing_address', 'tax_amount', 'shipping_cost',	'total_amount',	'currency',	'created_at',	'updated_at',	'subtotal'],errors='ignore',inplace= True)

In [ ]:
#aca limpio los datos que son Nan y no me aportan informacion sobre a que estado corresponde.
discounts_by_state = discounts_by_state.dropna(subset=['state'])


In [ ]:
discounts_by_state

In [ ]:
discounts_by_state['state'].unique()

In [ ]:
discounts_by_state

In [ ]:
discounts_by_state = discounts_by_state.groupby('state')['discount_amount'].agg(Total_discounts = 'sum', Average = 'mean',num_discounts=('count'))


In [ ]:
discounts_by_state

In [ ]:
max_total =  discounts_by_state.loc[discounts_by_state['Total_discounts'].idxmax()]


In [ ]:
max_total

In [ ]:
max_average = discounts_by_state.loc[discounts_by_state['Average'].idxmax()]

In [ ]:
max_average

In [ ]:
resume = pd.DataFrame([
    {'metrics': 'max_total','State': discounts_by_state['Total_discounts'].idxmax(),'value': discounts_by_state['Total_discounts'].max()},
    {'metrics': 'max_average','State': discounts_by_state['Average'].idxmax(),'value': discounts_by_state['Average'].max()}
    ])


In [ ]:
resume

In [ ]:
#compruebo si es correcto la informacion de resume
discounts_by_state.groupby('state')['Average'].max()

# 2¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado ‘Refunded’? ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

In [ ]:
#2-¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado ‘Refunded’?
# ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?
orders['zip_code'] = orders['shipping_address'].str.extract(r'(\d{5})')

refunded = orders[orders['status'] == 'Refunded']




In [ ]:
top5_zip = refunded['zip_code'].value_counts().head(5).index.tolist()
# conseguir los 5 zip en una lista

In [ ]:
refunded_top5 = refunded[refunded['zip_code'].isin(top5_zip)]
#me fijo si busco dentro esos codigos postales los refunded

In [ ]:
refunded_top5 = refunded_top5.merge(customers[['customer_id','first_name']],on = 'customer_id')
# merge de refunded_top 5 con customers para agregar la tabla de nombres.

In [ ]:
refunded_top5 = refunded_top5[refunded_top5['first_name'] != '-']

In [ ]:
#busco el nombre mas comun por zipcode
result = refunded_top5[refunded_top5['zip_code'].isin(top5_zip)].groupby('zip_code')['first_name'].agg(lambda x: x.mode()[0])

In [ ]:
result

# **Para cada tipo de pago y segmento de cliente, devolver la suma y el promedio expresado como porcentaje, de clientes activos y de consentimiento de marketing. Se valora que el output de la consulta tenga nombres claros y en español.**

In [ ]:
#3-Para cada tipo de pago y segmento de cliente, devolver la suma y el promedio expresado como porcentaje, de clientes activos y de consentimiento de marketing.
# Se valora que el output de la consulta tenga nombres claros y en español.

In [1]:
#CUSTOMER:customer_segment,is_active, marketing_consent
#ORDER:payment_method
payment_segment_stats = orders[['customer_id','payment_method']].drop_duplicates(['customer_id','payment_method']).merge(customers[['customer_id','customer_segment','marketing_consent','is_active']],on='customer_id',how='inner')
# payment_segment_stats = orders.merge(customers[['customer_id','customer_segment','is_active','marketing_consent']], on='customer_id', how='inner')


NameError: name 'orders' is not defined

In [ ]:
payment_segment_stats = payment_segment_stats.groupby(['customer_segment','payment_method']).agg(
     total_clientes = ('customer_id','nunique'),
     marketing = ('marketing_consent','mean'),
     clientes_activos = ('is_active','mean')
 )

In [ ]:
payment_segment_stats['% clientes_activos'] = 100 * payment_segment_stats['clientes_activos']
payment_segment_stats['% marketing'] = 100 * payment_segment_stats['marketing']

In [ ]:
payment_segment_stats = payment_segment_stats.drop(columns=['clientes_activos','marketing'])

In [ ]:
payment_segment_stats

# **SPara los productos que contienen en su descripción la palabra “stuff” (sin importar mayúsculas o minúsculas), calcular el peso total de su inventario agrupado por marca, mostrar sólo la marca y el peso total de las 5 más pesadas.**

In [ ]:
# #4Para los productos que contienen en su descripción la palabra “stuff” (sin importar mayúsculas o minúsculas),
#  calcular el peso total de su inventario agrupado por marca,
# mostrar sólo la marca y el peso total de las 5 más pesadas.


#product en descripcion str.contains('stuff'),:]

In [ ]:
products_stuff = products.loc[products['description'].str.lower().str.contains('stuff'),:]
products_stuff = products_stuff[products_stuff['brand'] != 'Undefined']
products_stuff ['inventory_weight'] = products_stuff['stock_quantity'] * products_stuff['weight_kg']


In [ ]:
products_stuff

In [ ]:
products_stuff = products_stuff.groupby('brand')['inventory_weight'].sum()

In [ ]:
products_stuff

In [ ]:
products_stuff = products_stuff.reset_index()

In [ ]:
products_stuff['total_tonnes'] = products_stuff['inventory_weight'] / 1_000

In [ ]:
products_stuff =products_stuff.sort_values('total_tonnes',ascending=False)

In [ ]:
products_stuff

In [ ]:
products_stuff.head(5)

# **CONSULTAS EXTRAS**

1-¿Cuál es el patrón de ventas a lo largo de los meses del año?

2- Top 10 promedio de rating por categoría de productos activos

3-¿Existe relación entre el precio del producto y la cantidad de votos útiles de sus reseñas?

4-¿Qué segmento de clientes realiza más compras?

5-¿Cuál es el país con mayor cantidad de clientes activos  y cuánto representan en el total?

6-¿Cuál es la cantidad de clientes activos por genero por país?

7-¿Cuál es la cantidad de clientes por genero que realizaron compras por país?

8-Distribución de reseñas por puntaje

# 1-¿Cuál es el patrón de ventas a lo largo de los meses del año?

In [ ]:
orders

In [ ]:
sales_completed = orders[(orders['status'] == 'Completed')]
sales_completed

In [ ]:
sales_completed.dropna(subset=["order_date", "created_at"],inplace= True)

In [ ]:
sales_completed

In [ ]:
#1- ¿En qué períodos (meses, días de la semana) se concentran más las ventas?
monthly_sales = sales_completed.groupby(sales_completed['order_date'].dt.month_name())['order_id'].count().reset_index()

monthly_sales = monthly_sales.rename(columns={'order_date':'Month','order_id':'Sales Mounth'})
monthly_sales


In [ ]:
monthly_sales = monthly_sales.sort_values('Month')


In [ ]:
monthly_sales

# 2- Top 10 promedio de rating por categoría de productos activos

In [ ]:
products

In [ ]:
categories

In [ ]:
reviews

In [ ]:
categories_in_order = categories.set_index(["parent_category","category_name"]).sort_index(ascending=True)

In [ ]:
categories_in_order

In [ ]:
categories_in_order = categories_in_order.reset_index()

In [ ]:
avg_rating_by_category = categories_in_order.merge(products[['product_id',	'product_name',	'category_id', 'is_active']],on='category_id',how='inner')




In [ ]:
avg_rating_by_category = avg_rating_by_category.merge(reviews[['product_id','rating','is_verified_purchase','helpful_votes','created_at']],on= 'product_id', how='inner')

In [ ]:
# voy a tomar solo los productos que esten activos en el mercado y que tengan una categoria principal asociada.
avg_rating_by_category = avg_rating_by_category[
    (avg_rating_by_category['is_active'] == True) &
    (avg_rating_by_category['parent_category'] != 'Unknown') &
    (avg_rating_by_category['category_name'] != 'Unknown')
]


In [ ]:
avg_rating_by_category

In [ ]:
avg_rating_by_category.dropna(subset=["rating"],inplace=True)
avg_rating_by_category.dropna(subset=["created_at"],inplace=True)

In [ ]:
avg_rating_by_category = avg_rating_by_category[avg_rating_by_category['is_active'] == True]

In [ ]:
avg_rating_by_category

In [ ]:
# aca saco los productos que no tienen una calificacion. ya que no me sirven para el analisis que se pretende hacer.
avg_rating_by_category = avg_rating_by_category.groupby(['parent_category', 'category_name']).agg(Promedio_Calificacion =('rating','mean')).dropna()

In [ ]:
avg_rating_by_category.rename(columns={'Promedio_Calificacion':'Average rating'},inplace= True)

In [ ]:
avg_rating_by_category

In [ ]:
avg_rating_by_category = avg_rating_by_category.reset_index().groupby('parent_category', as_index=False)['Average rating'].mean().nlargest(10, 'Average rating')

avg_rating_by_category['parent_category'] = avg_rating_by_category['parent_category'].astype(str)

In [ ]:
avg_rating_by_category

#3-¿Existe relación entre el precio del producto y la cantidad de votos útiles de sus reseñas?


In [ ]:
products

In [ ]:
reviews

In [ ]:
# verified reviews son aquellos clientes que realmente certificaron haber comprado el producto
verified_reviews = reviews[reviews['is_verified_purchase'] == True]


In [ ]:
verified_reviews.drop(columns = ['title','comment','is_verified_purchase'], errors = 'ignore',inplace =True) # sco estas col porque no me aportan informacion relevante para el analisis

In [ ]:
verified_reviews

In [ ]:
prod_reviews = products.drop(columns = ['stock_quantity','weight_kg','dimensions','description','0created_at','cost','created_at'], errors = 'ignore',inplace =False)

In [ ]:
# en este caso voy a tomar los productos que solo esten activos en el mercado,otro analisis que se puede hacer es productos que ya no estan activos, si tiene que ver con su precio y las reviews,osea si hay una correlacion entre ambas.
prod_active_reviews = prod_reviews[prod_reviews['is_active'] == True]

In [ ]:
prod_active_reviews = prod_active_reviews.merge(verified_reviews,on='product_id',how='inner')

In [ ]:
prod_active_reviews.drop(columns=['is_active'], errors= 'ignore', inplace = True)

In [ ]:
prod_active_reviews = prod_active_reviews.dropna(subset=['rating','price'])

In [ ]:
prod_active_reviews

In [ ]:
reviews_per_produtcs = prod_active_reviews.groupby(['product_id','price'])['review_id'].count().reset_index().rename(columns={"review_id":"n_reviews"})

In [ ]:
reviews_per_produtcs

In [ ]:
plt.figure(figsize=(10,6))
sns.regplot(
    data=prod_active_reviews,
    x="price", y="helpful_votes",
    scatter_kws={'alpha':0.4},   # transparencia de puntos
    line_kws={'color':'red'}     # estilo de la línea de regresión
)
plt.title("Price vs. useful votes (with linear regression)")
plt.xlabel("Product price")
plt.ylabel("Helpful votes in the review")
plt.show()


¿Hay relación entre el precio de los productos y el puntaje de las reseñas?

In [ ]:
prod_active_reviews[['price', 'helpful_votes']].corr()

In [ ]:
# plt.figure(figsize=(30,8))
# sns.scatterplot(
#     data=prod_active_reviews,
#     x="price", y="rating",
#     alpha=0.5
# )
# plt.title("Relación entre precio y rating")
# plt.xlabel("Precio del producto")
# plt.ylabel("Calificación (rating)")
# plt.show()

In [ ]:
# corr = prod_active_reviews[['price','rating']].corr(method='pearson').iloc[0,1]
# print(f"Correlación de Pearson: {corr:.3f}")

# 4-¿Qué segmento de clientes realiza más compras?

In [ ]:
customers

In [ ]:
orders

In [ ]:
orders_per_segment = orders[orders['status'] == 'Completed'].merge(customers[['customer_id','customer_segment']], on='customer_id',how='inner')

In [ ]:
orders_per_segment = orders_per_segment.groupby('customer_segment')['order_id'].nunique().sort_values(ascending=False).reset_index()

In [ ]:
orders_per_segment.rename(columns = {'order_id':'quantity of orders'},inplace= True)


In [ ]:
orders_per_segment


# 5-¿Cuál es el país con mayor cantidad de clientes activos  y cuánto representan en el total?

In [ ]:
active_customers = customers[customers['is_active'] == True]


In [ ]:
active_customers

In [ ]:
active_by_country = active_customers.groupby('country')['customer_id'].nunique().reset_index(name='num_active_customers').sort_values('num_active_customers', ascending=False)

In [ ]:
active_by_country = active_by_country[active_by_country['country']!='Undefined']
active_by_country

In [ ]:
total_costumers = active_by_country['num_active_customers'].sum()

In [ ]:
active_by_country['pct_of_total'] = (active_by_country['num_active_customers'] / total_costumers * 100).round(2)

In [ ]:
active_by_country

# 6-¿Cuál es la cantidad de clientes activos por genero por país?

In [ ]:
orders

In [ ]:
gender_country = customers[ (customers['country'] != 'Undefined') & (customers['is_active'] == True)]

In [ ]:
gender_country.country.unique()

In [ ]:
gender_country = gender_country.groupby(['country','gender'])['customer_id'].nunique().reset_index(name='num_customers').sort_values(['country','num_customers'], ascending=[True,False])




In [ ]:
gender_country = gender_country[gender_country['country'] != 'Undefined']

In [ ]:
gender_country

# 7-¿Cuál es la cantidad de clientes por genero que realizaron compras por país?

In [ ]:
orders.status.unique()

In [ ]:
customers

In [ ]:
valid_status = ['Completed','Shipped']
orders_valid = orders[orders['status'].isin(valid_status)]

In [ ]:
orders_valid = orders_valid.merge(customers[['country','customer_id','gender']],on='customer_id', how='inner')

In [ ]:
orders_valid = orders_valid[orders_valid['country'] != 'Undefined']

In [ ]:
orders_valid = orders_valid.dropna(subset=['order_date'])

In [ ]:
orders_valid = orders_valid.dropna(subset=['updated_at'])

In [ ]:
buyers_by_gender_country = orders_valid.groupby(['country','gender'])['customer_id'].nunique().reset_index(name='num_buyers').sort_values(['country','num_buyers'], ascending=[True,False])

In [ ]:
buyers_by_gender_country = buyers_by_gender_country [buyers_by_gender_country['country'] != 'Undefined']

In [ ]:
buyers_by_gender_country

# **8-Distribución de reseñas por puntaje**

In [ ]:
df = reviews[reviews['is_verified_purchase'] == True].dropna(subset=['created_at'])

In [ ]:
reviews_by_rating = df.groupby('rating')['review_id'].count().reset_index(name='num_reviews')

In [ ]:
reviews_by_rating

# **VISUALIZACIONES**

---



# **1)Una continua con una línea de tiempo**

pregunta 1: ¿Cuál es el patrón de ventas a lo largo de los meses del año?

In [ ]:
order_months = ["January", "February", "March", "April", "May", "June",
                "July", "August", "September", "October", "November", "December"]

monthly_sales["Month"] = pd.Categorical(monthly_sales["Month"],
                                        categories=order_months,
                                        ordered=True)

In [ ]:
plt.figure(figsize=(12,5))
sns.lineplot(data=monthly_sales, x='Month', y='Sales Mounth',markers='o')
plt.title("Monthly Sales Overview")
plt.ylim(0, monthly_sales["Sales Mounth"].max()*1.1)  # empieza en 0 y deja 10% de aire arriba
plt.grid(axis="y", linestyle="--", alpha=0.7)

for i, row in monthly_sales.iterrows():
    plt.vlines(x=row["Month"], ymin=0, ymax=row["Sales Mounth"], color="red", linestyle="--", alpha=0.5)

plt.xticks(rotation=45)
plt.show()


# **2) Una discreta con una continua.**



8-¿Cómo se distribuyen las calificaciones de los productos según su categoría principal?



In [ ]:
avg_rating_by_category.reset_index()

In [ ]:
plt.figure(figsize=(25,10))
sns.violinplot(data=reviews_with_cat,x="parent_category", y="rating", inner="quartile")   # muestra mediana y cuartiles
plt.xticks(rotation=90)
plt.title("Distribution of ratings by main category")
plt.xlabel('Main category')
plt.ylabel('Rating')

plt.show()


# **3)Una discreta con una discreta.**

6-¿Cuál es la cantidad de clientes activos por genero por país?

In [ ]:
gender_country

6 ¿Cuál es la cantidad de clientes activos por género y país?

In [ ]:
colors = ["#1b9e77", "#d95f02", "#7570b3"]
pivot = gender_country.pivot(index="country", columns="gender", values="num_customers")

pivot.plot(kind="bar", stacked=False, figsize=(30,5),color= colors)

plt.title("Active clients by gender and country")
plt.xlabel("Country")
plt.ylabel("Number of clients")
plt.xticks(rotation=45)
plt.legend(title="Gender", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.show()


# **4) Una continua con otra continua.**

scatter plot con pregunta 3 extras.

# **5) Un heatmap.**

Para cada tipo de pago y segmento de cliente clientes activos.Ver como se distribuye

In [ ]:
cust_method_counts = orders.groupby(['customer_id','payment_method']).size().reset_index(name='n_orders')


#Quedarse con el método más usado por cliente
preferred = cust_method_counts.sort_values(['customer_id','n_orders'], ascending=[True, False]).drop_duplicates('customer_id')  # 1 fila por cliente

#info del cliente
df_heat = preferred.merge(customers[['customer_id','customer_segment','is_active']],on='customer_id', how='left')

#la consigna pide clientes activos:
df_heat = df_heat[df_heat['is_active']]

df_heat = pd.crosstab(df_heat['payment_method'], df_heat['customer_segment'], dropna=False)

# Normalizar por columna → cada columna suma 100
df_heat = df_heat.div(df_heat.sum(axis=0), axis=1) * 100

plt.figure(figsize=(10,6))
sns.heatmap(df_heat, annot=True, fmt=".1f", cmap="Blues", vmin=0, vmax=100)
plt.title("Distribution of payment methods by segment (columns = 100%)")
plt.xlabel("Customer segment")
plt.ylabel("Payment method")
plt.show()

# === 6. Chequeo rápido ===
print(df_heat.sum(axis=0).round(1))


# **6) Dos visualizaciones a elección.**

 2- Top 10 promedio de rating por categoría de productos activos


In [ ]:
avg_rating_by_category

In [ ]:
plt.figure(figsize=(15,6))
plt.hlines(y=df_top10['parent_category'], xmin=0, xmax=df_top10['Average rating'])
plt.plot(df_top10['Average rating'], df_top10['parent_category'], 'o'); plt.tight_layout()



plt.title('Top 10 parent categories by average rating')
plt.xlabel('Average rating')
plt.ylabel('Parent category')
plt.xlim(3, 5)
plt.show()



4-¿Qué segmento de clientes (customer_segment) realiza más compras?

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
wedges, texts, autotexts = ax.pie(
    orders_per_segment['quantity of orders'],
    labels=orders_per_segment['customer_segment'],
    autopct=lambda p: f'{p:.1f}%',
    startangle=90, counterclock=False,
    colors=sns.color_palette('pastel')
)
# “donut”
centre = plt.Circle((0,0), 0.70, fc='white')
ax.add_artist(centre)
ax.set_title('Share of completed purchases by customer segment')
ax.axis('equal')
plt.show()


# **7) Una visualización que use una de las siguientes:

Treemap
Sankey
Una visu “joint” (ej JointGrid)
Una visu hecha a mano con gráficas personalizadas.**

5-¿Cuál es el país con mayor cantidad de clientes activos (is_active=True) y cuánto representan en el total?

In [ ]:
active_by_country

In [ ]:
plt.figure(figsize=(10,6))
colors = ["red", "pink", "green","violet", "yellow", "blue","purple","orange","skyblue"]
squarify.plot(
    sizes=active_by_country["num_active_customers"],
    label=active_by_country["country"].astype(str) + " (" + active_by_country["num_active_customers"].astype(str) + ") ",
    value = active_by_country['pct_of_total'].astype(str) + "%",
    color=colors,
    pad= False,
    alpha=0.5

)
plt.title("Active clients by country")
plt.axis("off")
plt.show()


# **VISUALIZACIONES EXTRAS**

# **CONSULTA VISU EXTRA 1:**

In [ ]:
products_stuff = products_stuff.reset_index()

In [ ]:
products_stuff['brand']= products_stuff['brand'].astype(str)

In [ ]:
products_stuff

In [ ]:
df_top5_weight = products_stuff.copy()

df_top5_weight = df_top5_weight[df_top5_weight['total_tonnes'] > 0].sort_values('total_tonnes', ascending=False).head(5)

sizes  = df_top5_weight['total_tonnes'].to_list()
labels = [f"{b} ({t:,.1f} t)" for b, t in zip(df_top5_weight['brand'], df_top5_weight['total_tonnes'])]

# colores (5)
cmap   = plt.get_cmap('tab10')
colors = ["blue", "red", "green","violet", "yellow"]

plt.figure(figsize=(10,6))
squarify.plot(sizes=sizes, label=labels, color=colors,
              pad=True, alpha=0.5, text_kwargs={'fontsize':15})
plt.title('Total inventory weight by brand (Top 5, tons) with the word "stuff" in the description.')
plt.axis('off'); plt.tight_layout(); plt.show()

# **CONSULTA VISU EXTRA 2:**

In [ ]:
# compradores únicos
buyers = orders['customer_id'].dropna().drop_duplicates()

# país + género de quienes compraron
df_contry_gender = customers.loc[customers['customer_id'].isin(buyers), ['customer_id','country','gender']].copy()
df_contry_gender['country'] = df_contry_gender['country'].astype(str)
df_contry_gender['gender']  = df_contry_gender['gender'].astype(str).str.upper()

# filtra categorías no deseadas
df_contry_gender = df_contry_gender[df_contry_gender['gender'].isin(['F','M','OTHER'])]
df_contry_gender = df_contry_gender[df_contry_gender['country'].str.lower()!='other']

# tabla y % por país
df_contry_gender = (df_contry_gender.groupby(['country','gender'])['customer_id'].nunique().reset_index(name='n'))

df_contry_gender = df_contry_gender.pivot(index='country', columns='gender', values='n').fillna(0).astype(int)
df_contry_gender = (df_contry_gender.div(df_contry_gender.sum(axis=1), axis=0) * 100).round(1)

plt.figure(figsize=(10,6))
sns.heatmap(df_contry_gender, annot=True, fmt='.1f', cmap='Blues',
            cbar_kws={'label':'% within the country'},
            vmin = 0,
            linewidths=0.5,
            vmax= 100)


plt.title('Unique customers who purchased by Country and Gender (%)', pad=10)

plt.xlabel('Gender')
plt.ylabel('Country')
plt.tight_layout()
plt.show()


# **CONSULTA VISU EXTRA 3: 9-Distribución de reseñas por puntaje**

In [ ]:
reviews_by_rating

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(
    data=reviews_by_rating,
    x=reviews_by_rating["rating"],
    y=reviews_by_rating["num_reviews"],
    color="purple"
)
plt.title("Distribution of reviews by rating")
plt.xlabel('Rating (1 to 5)')
plt.ylabel("Number of reviews")
plt.ylim(0, 55000)
plt.show()